In [2]:
from tree_sitter import Language, Parser

PY_LANGUAGE = Language('./build/my-languages.so', 'python')

python_parser = Parser()
python_parser.set_language(PY_LANGUAGE)

In [3]:
tree = python_parser.parse(bytes("""
def foo():
    if bar:
        baz()
""", "utf8"))

In [6]:
root_node = tree.root_node
assert root_node.type == 'module'
assert root_node.start_point == (1, 0)
assert root_node.end_point == (4,0)

function_node = root_node.children[0]
assert function_node.type == 'function_definition'
assert function_node.child_by_field_name('name').type == 'identifier'

function_name_node = function_node.children[1]
assert function_name_node.type == 'identifier'
assert function_name_node.start_point == (1, 4)
assert function_name_node.end_point == (1, 7)

assert root_node.sexp() == "(module " "(function_definition " "name: (identifier) " "parameters: (parameters) " "body: (block " "(if_statement " "condition: (identifier) " "consequence: (block " "(expression_statement (call " "function: (identifier) " "arguments: (argument_list))))))))"

In [11]:
cursor = tree.walk()

assert cursor.node.type == 'module'

print(cursor.node)
print(cursor.node.type)

cursor.goto_first_child()
assert cursor.node.type == 'function_definition'


assert cursor.goto_first_child()
assert cursor.node.type == 'def'

# Returns `False` because the `def` node has no children
assert not cursor.goto_first_child()

assert cursor.goto_next_sibling()
assert cursor.node.type == 'identifier'

assert cursor.goto_next_sibling()
assert cursor.node.type == 'parameters'

assert cursor.goto_parent()
assert cursor.node.type == 'function_definition'

<Node kind=module, start_point=(1, 0), end_point=(4, 0)>
module
